In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_946787/3509738684.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# from transformers import pipeline
# from transformers import AutoTokenizer

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import bamboolib as bam

import pickle

In [4]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,util
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


In [5]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [6]:
with open('df_2.pickle', 'rb') as handle:
    df_2 = pickle.load(handle)

In [7]:
df_2.reset_index(inplace=True,drop=True)

In [8]:
df_2['sentence']=df_2.apply(lambda x:'"'+x['Item Short Text']+'" was bought from "'+x['Vendor Name 1']+'" for a price of '+str(x['Process Unit Price']),axis=1)

In [9]:
df_2.head()

                        Vendor Name 1             Item Short Text  \
0  HONGKONG IPURE BIOLOGY CO.,LIMITED                     Red Dye   
1       JH CHEMICAL (HK) CO., LIMITED                    Red Lead   
2                       MAGNESIA GMBH          Magnesium Stearate   
3                 Q-PAC GROUP PTE LTD       Silica Gel 5gm A/B PE   
4          TACT CHEMIE S.E.A. PTE LTD  GP Thinner (18 litres/tin)   

   Process Unit Price     Main     Prime LOB  \
0            50.74875  NON ICT  REST OF LOBS   
1            48.52244  NON ICT  REST OF LOBS   
2            32.42630  NON ICT  REST OF LOBS   
3            60.00000  NON ICT  REST OF LOBS   
4            68.00000  NON ICT  REST OF LOBS   

                                            sentence  
0  "Red Dye" was bought from "HONGKONG IPURE BIOL...  
1  "Red Lead" was bought from "JH CHEMICAL (HK) C...  
2  "Magnesium Stearate" was bought from "MAGNESIA...  
3  "Silica Gel 5gm A/B PE" was bought from "Q-PAC...  
4  "GP Thinner (18 li

In [10]:
# embeddings = model.encode(df_2['sentence'])
# with open('embeddings.pickle', 'wb') as handle:
#     pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('embeddings.pickle', 'rb') as handle:
    embeddings = pickle.load(handle)

In [14]:
embeddings.shape

(607178, 384)

In [13]:
# # from sklearn.decomposition import PCA
# pca = PCA(n_components=50)
# X_pca = pca.fit_transform(embeddings)
# X_pca.shape

In [15]:
# result = TSNE().fit_transform(X_pca)
# with open('result.pickle', 'wb') as handle:
#     pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('result.pickle', 'rb') as handle:
    result = pickle.load(handle)

In [16]:
result.shape

(607178, 2)

In [17]:
result_df = pd.DataFrame(result, columns = ['x','y'])
result_df['Main']=df_2['Main']

In [18]:
# plt.scatter(result[:, 0], result[:, 1])
fig = px.scatter(x=result_df['x'], y=result_df['y'], color=result_df['Main'], opacity=0.8)
# fig.show()
fig.write_html("result.html")

# Classification

In [25]:
from sklearn.model_selection import train_test_split

In [23]:
embeddings.shape

(607178, 384)

In [21]:
df_2.head()

                        Vendor Name 1             Item Short Text  \
0  HONGKONG IPURE BIOLOGY CO.,LIMITED                     Red Dye   
1       JH CHEMICAL (HK) CO., LIMITED                    Red Lead   
2                       MAGNESIA GMBH          Magnesium Stearate   
3                 Q-PAC GROUP PTE LTD       Silica Gel 5gm A/B PE   
4          TACT CHEMIE S.E.A. PTE LTD  GP Thinner (18 litres/tin)   

   Process Unit Price     Main     Prime LOB  \
0            50.74875  NON ICT  REST OF LOBS   
1            48.52244  NON ICT  REST OF LOBS   
2            32.42630  NON ICT  REST OF LOBS   
3            60.00000  NON ICT  REST OF LOBS   
4            68.00000  NON ICT  REST OF LOBS   

                                            sentence  
0  "Red Dye" was bought from "HONGKONG IPURE BIOL...  
1  "Red Lead" was bought from "JH CHEMICAL (HK) C...  
2  "Magnesium Stearate" was bought from "MAGNESIA...  
3  "Silica Gel 5gm A/B PE" was bought from "Q-PAC...  
4  "GP Thinner (18 li

In [29]:
X_train, X_test,y_train,y_test = train_test_split(embeddings,df_2['Main'],test_size=0.33, random_state=42)

In [32]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((406809, 384), (200369, 384), (406809,), (200369,))

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [38]:
y_train.value_counts(),y_test.value_counts()

(NON ICT    384984
 ICT         21825
 Name: Main, dtype: int64,
 NON ICT    189324
 ICT         11045
 Name: Main, dtype: int64)

In [39]:
model=RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

RandomForestClassifier()

In [40]:
y_pred=model.predict(X_test)

In [44]:
from sklearn import metrics

In [45]:
metrics.accuracy_score(y_test, y_pred)

0.9823076423997724

In [47]:
metrics.f1_score(y_test,y_pred,average='macro')

0.9000914739114784

In [48]:
X_train

array([[-0.0774722 , -0.01603771,  0.01099822, ..., -0.13258313,
        -0.101745  ,  0.03495754],
       [-0.09138422,  0.11162391,  0.00294701, ..., -0.165875  ,
        -0.00559152,  0.07580931],
       [-0.07254615,  0.03456019, -0.05872216, ..., -0.08735807,
        -0.1001168 ,  0.0315308 ],
       ...,
       [-0.01092109, -0.06297019, -0.01446409, ..., -0.13347471,
        -0.10098429,  0.07682315],
       [-0.12767811, -0.01132574,  0.06046413, ..., -0.13006252,
         0.04364389,  0.03651832],
       [-0.11053848,  0.01227363, -0.06568097, ..., -0.10385811,
        -0.00334514,  0.02962996]], dtype=float32)

In [56]:
y_train

22311     NON ICT
421742    NON ICT
221919    NON ICT
496543    NON ICT
367383    NON ICT
           ...   
110268    NON ICT
259178    NON ICT
365838    NON ICT
131932    NON ICT
121958    NON ICT
Name: Main, Length: 406809, dtype: object